In [1]:
pip install lm-format-enforcer


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install --upgrade --force-reinstall lm-format-enforcer



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached lm_format_enforcer-0.10.11-py3-none-any.whl.metadata (17 kB)
  Using cached interegular-0.3.3-py37-none-any.whl.metadata (3.0 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.27.2-cp312-cp312-win_amd64.whl.metadata (6.7 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached lm_format_enforcer-0.10.11-py3-none-any.whl (44 kB)
Using cached interegular-0.3.3-py37-none-any.whl (23 kB)
Using cached pydantic-2.10.6-py3-none-any.whl (431 kB)
Using cached pydantic_core-2.27.2-cp312-cp312-win_amd64.whl (2.0 MB)
Using cached packaging-24.2-py3-none-any.whl (65 kB)
Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl (156 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using 

In [2]:
pip list | grep lm-format-enforcer

Note: you may need to restart the kernel to use updated packages.


'grep' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
pip list


Package                                  Version
---------------------------------------- -----------
absl-py                                  2.1.0
aiohappyeyeballs                         2.4.6
aiohttp                                  3.11.12
aiosignal                                1.3.2
altair                                   5.5.0
annotated-types                          0.7.0
anyio                                    4.8.0
asttokens                                3.0.0
attrs                                    25.1.0
beautifulsoup4                           4.13.3
bert-score                               0.3.13
blinker                                  1.9.0
cachetools                               5.5.2
certifi                                  2025.1.31
charset-normalizer                       3.4.1
click                                    8.1.8
colorama                                 0.4.6
comm                                     0.2.2
contourpy                                1.

In [2]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer
from deepeval.models import DeepEvalBaseLLM
from pydantic import BaseModel
from lm_format_enforcer import enforce_format

class CustomBartSummarizer(DeepEvalBaseLLM):
    def __init__(self, model_name: str):
        self.model_name = model_name
        self.model = None
        self.tokenizer = None

    def load_model(self):
        if self.model is None or self.tokenizer is None:
            self.model = BartForConditionalGeneration.from_pretrained(
                self.model_name,
                torch_dtype=torch.float16
            ).to('cuda' if torch.cuda.is_available() else 'cpu')
            self.tokenizer = BartTokenizer.from_pretrained(self.model_name)
        return self.model

    def generate(self, prompt: str, schema: BaseModel) -> BaseModel:
        model = self.load_model()
        inputs = self.tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True).to(model.device)
        summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=150, early_stopping=True)
        raw_output = self.tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        
        # Enforce JSON format based on the provided schema
        enforced_output = enforce_format(raw_output, schema)
        return enforced_output

    async def a_generate(self, prompt: str, schema: BaseModel) -> BaseModel:
        return self.generate(prompt, schema)

    def get_model_name(self):
        return self.model_name


d:\LLM_TESTING_NEW\llmenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\LLM_TESTING_NEW\llmenv\Lib\site-packages\deepeval\__init__.py:53: UserWarning: You are using deepeval version 2.4.5, however version 2.4.6 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


ModuleNotFoundError: No module named 'lm_format_enforcer'